# Board games recommender system

### Imports and data loading

In [45]:
import numpy as np
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
import os

In [46]:
path = kagglehub.dataset_download("mshepherd/board-games")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\pauli\.cache\kagglehub\datasets\mshepherd\board-games\versions\13


In [47]:
csv_path = os.path.join(path, "bgg_GameItem.csv")
df = pd.read_csv(csv_path)
df.head(5)

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,stddev_rating,bayes_rating,complexity,language_dependency,bga_id,dbpedia_id,luding_id,spielen_id,wikidata_id,wikipedia_id
0,1,Die Macher,1986.0,5497,1,"12517,4959","133,2,24883,2726,15108,39249,11652,5382,8147",3.0,5.0,4.0,...,1.57510,7.06367,4.3144,1.166667,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dragonmaster,1981.0,5497,8384,12424,"64,20",3.0,4.0,3.0,...,1.46752,5.76599,1.9636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Samurai,1998.0,5497,2,11883,"17,133,267,29,7340,7335,41,2973,4617,1391,8291...",2.0,4.0,2.0,...,1.19279,7.22869,2.4784,1.006579,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Tal der Könige,1992.0,5497,8008,2277,37,2.0,4.0,2.0,...,1.23667,5.66660,2.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Acquire,1964.0,5497,4,"12658,18317","92,5,4871,3082,858,2962,28072,5392,4668,38,462...",2.0,6.0,3.0,...,1.33603,7.12541,2.4985,1.095890,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
print(df['bgg_id'].max())

392146


In [49]:
df['bgg_id'].count()

np.int64(113904)

In [50]:
df['bgg_id'].isna().sum()

np.int64(0)

In [51]:
df.columns

Index(['bgg_id', 'name', 'year', 'game_type', 'designer', 'artist',
       'publisher', 'min_players', 'max_players', 'min_players_rec',
       'max_players_rec', 'min_players_best', 'max_players_best', 'min_age',
       'min_age_rec', 'min_time', 'max_time', 'category', 'mechanic',
       'cooperative', 'compilation', 'compilation_of', 'family',
       'implementation', 'integration', 'rank', 'num_votes', 'avg_rating',
       'stddev_rating', 'bayes_rating', 'complexity', 'language_dependency',
       'bga_id', 'dbpedia_id', 'luding_id', 'spielen_id', 'wikidata_id',
       'wikipedia_id'],
      dtype='object')

### BGG Data Columns Description

In [52]:
csv_path = os.path.join(path, "bgg_Category.csv")
df_bgg_Category = pd.read_csv(csv_path)
df_bgg_Category.head(5)

,bgg_id,name
0,1001,Political
1,1002,Card Game
2,1008,Nautical
3,1009,Abstract Strategy
4,1010,Fantasy


In [53]:
csv_path = os.path.join(path, "bgg_GameFamily.csv")
df_bgg_GameFamily = pd.read_csv(csv_path)
df_bgg_GameFamily.head(5)

,bgg_id,name
0,1,Admin: Test Family
1,2,Game: Carcassonne
2,3,Game: Catan
3,4,Series: The Chicken Family of Zoch
4,5,Game: Bohnanza


In [54]:
csv_path = os.path.join(path, "bgg_GameType.csv")
df_bgg_GameType = pd.read_csv(csv_path)
df_bgg_GameType.head(5)

,bgg_id,name
0,4415,Amiga
1,4420,Commodore 64
2,4664,War Game
3,4665,Children's Game
4,4666,Abstract Game


In [55]:
csv_path = os.path.join(path, "bgg_Mechanic.csv")
df_bgg_Mechanic = pd.read_csv(csv_path)
df_bgg_Mechanic.head(5)

,bgg_id,name
0,2001,Action Points
1,2002,Tile Placement
2,2003,Rock-Paper-Scissors
3,2004,Set Collection
4,2005,Stock Holding


In [56]:
csv_path = os.path.join(path, "bgg_Person.csv")
df_bgg_Person = pd.read_csv(csv_path)
df_bgg_Person.head(5)

,bgg_id,name
0,1,Karl-Heinz Schmiel
1,2,Reiner Knizia
2,3,(Uncredited)
3,4,Sid Sackson
4,5,Jean du Poël


In [57]:
csv_path = os.path.join(path, "bgg_Publisher.csv")
df_bgg_Publisher = pd.read_csv(csv_path)
df_bgg_Publisher.head(5)

,bgg_id,name
0,2,Moskito Spiele
1,3,Rio Grande Games
2,4,(Self-Published)
3,5,The Avalon Hill Game Co
4,6,Historien Spiele Galerie (Historien Spielegale...


### BGG Data Columns Description

| Column | Description |
|--------|-------------|
| `bgg_id` | Unique game identifier in BGG database. |
| `name` | Game name (primary). |
| `year` | Game publication year. |
| `game_type` | Game type (board game, expansion, etc.). From main element. |
| `designer` | Game designer(s). |
| `artist` | Artist(s) responsible for artwork. |
| `publisher` | Game publisher. |
| `min_players` | Minimum number of players. |
| `max_players` | Maximum number of players. |
| `min_players_rec` | Minimum recommended number of players. From user polls. |
| `max_players_rec` | Maximum recommended number of players. From user polls. |
| `min_players_best` | Minimum number of players for best experience. From polls - "Best" votes. |
| `max_players_best` | Maximum number of players for best experience. From polls - "Best" votes. |
| `min_age` | Official minimum player age. |
| `min_age_rec` | Community recommended minimum age. From polls. |
| `min_time` | Minimum game time (minutes). |
| `max_time` | Maximum game time (minutes). |
| `category` | Game category. |
| `mechanic` | Game mechanics. |
| `cooperative` | Whether the game is cooperative. Probably boolean flag based on mechanics. |
| `compilation` | Whether the game is a compilation of other games. Based on tags or special designations. |
| `compilation_of` | Which games it is a compilation of. Details from game relationships. |
| `family` | Game family it belongs to. |
| `implementation` | Game implementations (digital, other versions). |
| `integration` | Integrations with other games. Relationships between games in BGG system. |
| `rank` | Position in BGG ranking. Data from separate API endpoint (not visible in this XML). |
| `num_votes` | Number of votes/ratings. From statistical data (not in this XML). |
| `avg_rating` | Average rating. From BGG statistical data. |
| `stddev_rating` | Standard deviation of ratings. From BGG statistical data. |
| `bayes_rating` | Bayesian rating (considers number of votes). Used for BGG rankings. |
| `complexity` | Game complexity level (1-5). From separate user polls. |
| `language_dependency` | Language dependency (1-5).<br>**1**: No text in the game<br>**2**: Little text - easy to memorize<br>**3**: Moderate text - cheat sheet needed<br>**4**: Lots of text - translation required<br>**5**: Unplayable in other language |
| `image` | Game image URL. |
| `description` | Game description. |

In [58]:
# Create dictionaries with string keys (to match the JSON parsing approach)
def create_string_key_dict(df, key_col, value_col):
    return {str(k).strip(): str(v).strip() for k, v in df.set_index(key_col)[value_col].to_dict().items()}

game_type_dict = create_string_key_dict(df_bgg_GameType, 'bgg_id', 'name')
designer_dict = create_string_key_dict(df_bgg_Person, 'bgg_id', 'name')
artist_dict = create_string_key_dict(df_bgg_Person, 'bgg_id', 'name')  # Note: should this be df_bgg_Artist?
publisher_dict = create_string_key_dict(df_bgg_Publisher, 'bgg_id', 'name')
category_dict = create_string_key_dict(df_bgg_Category, 'bgg_id', 'name')
mechanic_dict = create_string_key_dict(df_bgg_Mechanic, 'bgg_id', 'name')

In [59]:
ref_dicts = {'game_types': game_type_dict,
            'designers': designer_dict,
            'artists': artist_dict,
            'publishers': publisher_dict,
            'categories': category_dict,
            'mechanics': mechanic_dict}

In [60]:
# Check game types for this board game
game_types = df.loc[df['name']=='Gloomhaven', 'game_type'].values
game_types = game_types[0].split(',')
print('Game Types\n------------')
for type_id in game_types:
    print(game_type_dict[type_id])

# the output should be:
# Game Types
# ------------
# thematic
# strategygames

Game Types
------------
Thematic
Strategy Game


In [61]:
# Check categories for this board game
categories = df.loc[df['name']=='Gloomhaven', 'category'].values
categories = categories[0].split(',')
print('Categories\n------------')
for c_id in categories:
    print(category_dict[c_id])

Categories
------------
Adventure
Exploration
Fantasy
Fighting
Miniatures


In [62]:
# Check mechanics for this board game
mechanics = df.loc[df['name']=='Gloomhaven', 'mechanic'].values
mechanics = mechanics[0].split(',')
print('Mechanics\n------------')
for c_id in mechanics:
    print(mechanic_dict[c_id])

Mechanics
------------
Action Queue
Action Retrieval
Campaign / Battle Card Driven
Card Play Conflict Resolution
Communication Limits
Cooperative Game
Critical Hits and Failures
Deck Construction
Grid Movement
Hand Management
Hexagon Grid
Legacy Game
Line of Sight
Modular Board
Once-Per-Game Abilities
Role Playing
Scenario / Mission / Campaign Game
Simultaneous Action Selection
Solo / Solitaire Game
Tags
Variable Player Powers


In [63]:
# Remove the columns for other website ids
df.drop(columns=['bga_id', 'dbpedia_id', 'luding_id', 'spielen_id', 'wikidata_id', 'wikipedia_id'], inplace=True)

In [64]:
df_img_desc = pd.read_csv("boardgame_images_merged.csv")

In [65]:
df_img_desc

,bgg_id,image,description
0,1,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,Die Macher is a game about seven sequential po...
1,2,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,Dragonmaster is a trick-taking card game based...
2,3,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,Samurai is set in medieval Japan. Players comp...
3,4,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,When you see the triangular box and the luxuri...
4,5,https://cf.geekdo-images.com/gIQlWhlaqjPiunZjj...,"In Acquire, each player strategically invests ..."
...,...,...,...
113899,332492,NaN,NaN
113900,332493,NaN,Manhattan is a unification game that always en...
113901,332495,NaN,NaN
113902,332497,NaN,"In this game, at least four players try to get..."
